In [ ]:
from pathlib import Path

source_output_folder_base = Path('<Enter path to download root target folder>')
aligned_transcript_filename = 'transcript.aligned.json'
source_entry_ids = [i.name for i in source_output_folder_base.iterdir() if i.is_dir()]

In [ ]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

buckets = {}

total_duration = 0
total_segments_duration = 0
for source_entry_id in tqdm(source_entry_ids):
      metadata_path = source_output_folder_base / str(source_entry_id) / "metadata.json"
      with open(metadata_path, 'r') as f:
        metadata_content = json.loads(f.read())
      segments_metadata = metadata_content.get("per_segment_quality_scores", [])
      total_segments_duration += sum([s["end"] - s["start"] for s in segments_metadata])
      total_duration += metadata_content["session_duration"]

print(f"Total segments duration: {total_segments_duration} seconds")
print(f"Total duration: {total_duration} seconds")

100%|██████████| 5499/5499 [00:01<00:00, 5084.74it/s]

Total segments duration: 51747.55600000007 seconds
Total duration: 286853.2922448972 seconds


In [2]:
import stable_whisper
import numpy as np
import pandas as pd
from tqdm import tqdm

buckets = {}

for source_entry_id in tqdm(source_entry_ids):
    try:
        aligned_results_path = source_output_folder_base / str(source_entry_id) / aligned_transcript_filename
        aligned_results = stable_whisper.WhisperResult(str(aligned_results_path))
        
        segment_durations = [s.end - s.start for s in aligned_results.segments]
        word_probs_per_segment = [[w.probability for w in s.words] for s in aligned_results.segments]
        median_probs_per_segment = [np.median(probs) for probs in word_probs_per_segment]

        round_probs = (np.array(median_probs_per_segment) * 10).round() / 10

        for prob_bucket, duration in zip(round_probs, segment_durations):
            buckets[prob_bucket] = buckets.get(prob_bucket, 0)
            buckets[prob_bucket] += duration
    except:
        print('Did not work out - ', source_entry_id)

    

durations_per_prob_df = pd.DataFrame.from_dict(buckets, "index").sort_index(ascending=False)
durations_per_prob_df["cs"] = durations_per_prob_df.cumsum()
durations_per_prob_df["csp"] = durations_per_prob_df["cs"] / durations_per_prob_df["cs"].iloc[-1]
durations_per_prob_df

 44%|████▍     | 2430/5500 [00:05<00:06, 511.61it/s]

Did not work out -  manifest.csv


100%|██████████| 5500/5500 [00:12<00:00, 452.88it/s]


,0,cs,csp
1.0,1.240,1.240,0.000024
0.9,20.240,21.480,0.000415
0.8,66.040,87.520,0.001691
0.7,125.870,213.390,0.004124
0.6,296.670,510.060,0.009857
0.5,886.165,1396.225,0.026981
0.4,2227.948,3624.173,0.070036
0.3,5373.990,8998.163,0.173886
0.2,10815.479,19813.642,0.382890
0.1,19156.585,38970.227,0.753083
